In [1]:
import cv2
from cv2 import aruco
import numpy as np
import pandas as pd
import os
import glob

In [2]:
# ArUco dictionary generate
aruco_dict = aruco.Dictionary_get(aruco.DICT_5X5_100)
# random marker board generate
board = aruco.CharucoBoard_create(7, 6, 1, .8, aruco_dict)
# board image generate
imboard = board.draw((2000, 2000))

# calibrate camera parameter
calibration_flags = (cv2.CALIB_USE_INTRINSIC_GUESS +
             cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_FIX_ASPECT_RATIO)
calibration_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-9)


In [3]:
# Extract individual images stored in a given directory
images = glob.glob('./Calibration_dataset/*.jpg')
captured_images = []

In [4]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

# Find checkerboard 2D coordinates

In [5]:
allCorners = []
allIds = []

decimator = 0
criteria = (cv2.TERM_CRITERIA_EPS +
                cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(
            gray, aruco_dict)

    if len(corners) > 0:
        # SUB PIXEL DETECTION
        for corner in corners:
            # Checkerboard corner improvements
            cv2.cornerSubPix(gray, corner,
                                winSize=(3, 3),
                                zeroZone=(-1, -1),
                                criteria=criteria)
        res2 = cv2.aruco.interpolateCornersCharuco(
            corners, ids, gray, board)
        if res2[1] is not None and res2[2] is not None and len(res2[1]) > 3 and decimator % 1 == 0:
            allCorners.append(res2[1])
            allIds.append(res2[2])

    decimator += 1
imsize = gray.shape


# Camera calibration

In [6]:
cameraMatrixInit = np.array([[1000.,    0., imsize[0]/2.],
                                 [0., 1000., imsize[1]/2.],
                                 [0.,    0.,           1.]])

distCoeffsInit = np.zeros((5, 1))
flags = (cv2.CALIB_USE_INTRINSIC_GUESS +
         cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_FIX_ASPECT_RATIO)
# flags = (cv2.CALIB_RATIONAL_MODEL)
(ret, camera_matrix, distortion_coefficients0,
rotation_vectors, translation_vectors,
stdDeviationsIntrinsics, stdDeviationsExtrinsics,
perViewErrors) = cv2.aruco.calibrateCameraCharucoExtended(
    charucoCorners=allCorners,
    charucoIds=allIds,
    board=board,
    imageSize=imsize,
    cameraMatrix=cameraMatrixInit,
    distCoeffs=distCoeffsInit,
    flags=calibration_flags,
    criteria=calibration_criteria)

## Intrinsic camera matrix

In [7]:
print("Camera matrix : \n") 
print(camera_matrix)

Camera matrix : 

[[7.64751969e+02 0.00000000e+00 1.04080185e+03]
 [0.00000000e+00 7.64751969e+02 1.17123608e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


## Lens distortion coefficients

In [8]:
print("Distortion coefficients : \n") 
print(distortion_coefficients0)

Distortion coefficients : 

[[-0.00234146]
 [-0.00943689]
 [-0.03305267]
 [-0.00434785]
 [-0.02999369]
 [ 0.01724588]
 [ 0.01557708]
 [-0.03983207]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


## Rotation matrix

In [11]:
print("rvecs : \n") 
print(rotation_vectors)

rvecs : 

(array([[ 3.0498011 ],
       [ 0.0307864 ],
       [-0.03294996]]), array([[ 3.0464983 ],
       [-0.00684857],
       [-0.02135296]]), array([[ 3.05297099],
       [-0.00354083],
       [-0.02254089]]), array([[ 3.05843832e+00],
       [-2.83536528e-03],
       [-2.18790942e-02]]), array([[ 3.0499601 ],
       [ 0.00696822],
       [-0.02989532]]), array([[ 3.05203287],
       [-0.01246835],
       [-0.01293518]]), array([[ 3.05030213],
       [-0.00694365],
       [-0.02110914]]), array([[ 3.04763954],
       [-0.00608836],
       [-0.0158901 ]]), array([[ 3.050597  ],
       [-0.01322168],
       [-0.0132542 ]]), array([[ 3.04983506e+00],
       [ 9.65933717e-04],
       [-2.48092945e-02]]), array([[ 3.04874973],
       [-0.01078716],
       [-0.01368802]]), array([[ 3.05013752],
       [ 0.03039838],
       [-0.03065611]]), array([[ 3.0448228 ],
       [-0.01188953],
       [-0.01244924]]), array([[ 3.04646758],
       [ 0.01103063],
       [-0.03383788]]), array([[ 3.04

## Translation matrix

In [12]:
print("tvecs : \n")
print(translation_vectors)

tvecs : 

(array([[-6.00168178],
       [-4.48152065],
       [ 7.37432115]]), array([[-6.76423768],
       [-4.16620414],
       [ 7.35435014]]), array([[-6.54864085],
       [-4.64261255],
       [ 7.32501926]]), array([[-6.49740507],
       [-5.01119372],
       [ 7.37970508]]), array([[-6.39231817],
       [-4.73744772],
       [ 7.42153439]]), array([[-7.02421039],
       [-4.5958977 ],
       [ 7.33095697]]), array([[-6.29766609],
       [-4.5707929 ],
       [ 7.44144877]]), array([[-6.81584443],
       [-4.49765047],
       [ 7.40557638]]), array([[-6.63400953],
       [-4.58334358],
       [ 7.44154692]]), array([[-6.20834748],
       [-4.59760779],
       [ 7.45160188]]), array([[-6.72680669],
       [-4.48155276],
       [ 7.40385601]]), array([[-5.78532068],
       [-4.57239676],
       [ 7.39330538]]), array([[-6.56920791],
       [-4.38100283],
       [ 7.37675244]]), array([[-6.06246109],
       [-4.40700164],
       [ 7.44879758]]), array([[-5.92684262],
       [-4.3640

# Undistort

In [10]:
# undistort
dir="Calibration_dataset"
createDirectory("undistort")

for i in os.listdir(dir):
    img=cv2.imread(dir+'/'+i)
    img_undist = cv2.undistort(img, camera_matrix, distortion_coefficients0, None)
    concatenated_image = cv2.hconcat([img, img_undist])
    concatenated_image = cv2.resize(concatenated_image, (0,0), fx=0.5, fy=0.5)
    cv2.imshow("undistorted", concatenated_image)
    cv2.imwrite("undistort/undist_{0}.png".format(i), concatenated_image)
    key = cv2.waitKey(0)
    if key == 27:
        cv2.destroyAllWindows()